In [ ]:
import numpy as np
import math as m
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy.stats import norm
import yfinance as yf
from dateutil import parser
from datetime import datetime
from pandas_datareader.data import DataReader as dr
from scipy.interpolate import CubicSpline
%matplotlib inline

In [ ]:
ticker = 'MSFT' # set stock to be analysed, e.g. MSFT = Microsoft Corporation
stock = yf.Ticker('MSFT') # Pull information from Yahoo Finance

In [ ]:
Maturity = stock.options  # pull the time to maturities for the option
today = datetime.today().date() # set todays date
maturityDates = np.zeros(len(Maturity))
# Process maturity dates to get in terms of 1 year 
for i in range(len(Maturity)):
    t = parser.parse(Maturity[i]).date() - today
    maturityDates[i] = t.days/365 

In [ ]:
# Set the strikes so they are the same for each maturity
maxStrike = np.zeros(len(Maturity)) # placeholder
minStrike = np.zeros(len(Maturity)) # placeholder
for i in range(len(Maturity)):
    maxStrike[i] = stock.option_chain(Maturity[i]).calls.iloc[:,2].max() # Find the max strike price at each maturity
    minStrike[i] = stock.option_chain(Maturity[i]).calls.iloc[:,2].min() # Find the minimum strike price ate each maturity
overallMax = maxStrike.min() # Find the largest strike price each maturity has in common
overallMin = minStrike.max() # Find the smallest strike price each maturity has in common

In [ ]:
NewStrike = np.arange(overallMin, overallMax + 1) # create a new array for strike prices

In [ ]:
yahoo_IV = pd.DataFrame(columns=Maturity) # Dataframe for implied volatility
# Use the implied volitility data provided in yahoo finance
for i in range(len(Maturity)):
# Uncomment below if you want to interpolate the implied volatility linearly
#     yahoo_IV[Maturity[i]] = np.interp(NewStrike, tsla.option_chain(Maturity[i]).calls.iloc[:,2],
#                                      tsla.option_chain(Maturity[i]).calls.iloc[:,10])

# Uncomment below if you want to interpolate the implied volatility using a cubic spline
    YahooSpline = CubicSpline(stock.option_chain(Maturity[i]).calls.iloc[:,2],
                                       stock.option_chain(Maturity[i]).calls.iloc[:,10], bc_type='natural')
    yahoo_IV[Maturity[i]] = YahooSpline(NewStrike)
yahoo_IV.set_index(NewStrike) 

In [ ]:
# plot volatility surface
yahoo_IV_np = yahoo_IV.to_numpy()
# 3D surface plot of matrix V
fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection='3d')
X, Y =np.meshgrid(maturityDates, NewStrike)
s = ax.plot_surface(X, Y, yahoo_IV_np, cmap = 'Spectral')
ax.set_xlabel('Time to Maturity', fontsize=14)
ax.set_ylabel('Strike Price', fontsize=14)
ax.set_zlabel('Implied Volatility', fontsize=14)
ax.xaxis.set_tick_params(labelsize=12)
ax.yaxis.set_tick_params(labelsize=12)
ax.zaxis.set_tick_params(labelsize=12)
# ax.set_title('Implied Volatility from Yahoo', fontsize=16)
fig.colorbar(s, shrink=0.5, aspect=20)
ax.view_init(22.5,45)
plt.show()